In [40]:
#modules
from keras.models import load_model


In [41]:
MODEL = load_model("CNN_MODEL.h5")


In [42]:
import numpy as np
import os
import cv2
from scipy.ndimage import convolve
from keras.utils import to_categorical, plot_model
import matplotlib.pyplot as plt

In [43]:
img_size1, img_size2 = 96,96
vertical_prewitt_filter = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
horizontal_prewitt_filter = np.array([[-1,-1,-1],[0,0,0],[1,1,1]])
def load_data(path, train=True):
    print("Loading data from: ", path)
    data = []
    for img in os.listdir(path):
        imgname, ext = os.path.splitext(img)
        ID, etc = imgname.split('__')
        ID = int(ID) - 1
        if train:
            _, lr, finger, _, _ = etc.split('_')
        else:
            _, lr, finger, _  = etc.split('_')
        if lr=='Left':
            base = 0
        else: base  = 5
        if finger=="little":
            fingerNum = base + 0
        elif finger=='ring':
            fingerNum = base + 1
        elif finger=='middle':
            fingerNum = base + 2
        elif finger=='index':
            fingerNum = base + 3 
        else: fingerNum = base + 4
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        gauss_blur = cv2.GaussianBlur(img_array,(1,1),0)
        _,th1 = cv2.threshold(gauss_blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        convolved_img =  convolve(th1,vertical_prewitt_filter )
        convolved_img =  convolve(convolved_img,horizontal_prewitt_filter )
        img_resize = cv2.resize(img_array, (img_size1, img_size2))
        data.append([ID, fingerNum, img_resize])
    return data

In [44]:
user_path = "C:\\ONE_DRIVE_DATA\\Desktop\\GRIET\\AAC\\PROJECT\\INPUT\\TEST\\USER_INP"
user_data = load_data(user_path, train=True)

Loading data from:  C:\ONE_DRIVE_DATA\Desktop\GRIET\AAC\PROJECT\INPUT\TEST\USER_INP


In [45]:
DB_path = "C:\\ONE_DRIVE_DATA\\Desktop\\GRIET\\AAC\\PROJECT\\INPUT\\TEST\\DB"
db_datas = load_data(DB_path,train = False)

Loading data from:  C:\ONE_DRIVE_DATA\Desktop\GRIET\AAC\PROJECT\INPUT\TEST\DB


In [46]:
#TRUE INPUT DATA
img_size1, img_size2 = 96,96
inp_data = user_data[5]
print(inp_data)
inp_features = np.array(inp_data[2]).reshape(-1, img_size1, img_size2, 1)
inp_features = inp_features/255.0
#print(inp_features)
inp_id = to_categorical(inp_data[0],num_classes = 10) #to_categorical([db_data[0],], num_classes=600)
print(inp_id)


[8, 8, array([[160, 158, 158, ...,   0,   0,   0],
       [160, 105, 121, ...,   0,   0,   0],
       [160, 105, 255, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [47]:
rand_fp_num = 0
#plt.imshow(inp_features.reshape((906, 1577)), cmap ='gray')

y_SubjectID_pred = MODEL.predict(inp_features)
Id_pred = np.argmax(y_SubjectID_pred)

print('PREDICTED ID : ',Id_pred)

#print(y_SubjectID_pred)
#print(y_SubjectID_test )

dec = 0
for db_data in db_datas:
    db_features = np.array(db_data[2]).reshape(-1, 96, 96, 1)
    db_features = db_features/255.0
    db_ID = to_categorical([db_data[0],], num_classes=600)
    y_SubjectID_test = db_ID
    Id_real = np.argmax(y_SubjectID_test)
    print('DATABASE ID : ',Id_real)
    if Id_pred==Id_real:
        print("Infomation confirm! Fingerprint matches: person Id",Id_pred)

        cv2.waitKey(0)
        cv2.destroyAllWindows()

        dec = 1
        break
if dec == 0 :
    print("PERSON DOES NOT EXIST IN DATABASE")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
PREDICTED ID :  8
DATABASE ID :  12
DATABASE ID :  15
DATABASE ID :  0
DATABASE ID :  0
DATABASE ID :  0
DATABASE ID :  8
Infomation confirm! Fingerprint matches: person Id 8
